In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [5]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\IL-1β")

In [6]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [7]:
Pubchem_data=pd.read_csv("./Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [8]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [9]:
#Load the data
Raw_data= pd.read_csv('./Original_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Scaler_transformer.pkl')

In [10]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7279 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [11]:
X_scaled_data

,ESOL,LOGPcons,LOGP99,MRcons,MR99,SM5_B(p),SM6_B(p),SM3_B(p),SM4_B(p),H_Dz(p),...,C-006,Chi_Dz(p),Eta_D_epsiB,Eig01_AEA(ed),SpMax_AEA(ed),GATS7e,SpDiam_AEA(ri),MATS1i,F06[N-N],s3_phRelSize_2
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
53300413.0,0.374972,0.693125,0.704468,0.838102,0.838102,0.863458,0.843964,0.898451,0.885361,0.756514,...,0.411765,0.043701,0.174087,0.301063,0.301063,0.486872,0.305242,0.514046,0.3,0.217461
49795501.0,0.483465,0.655272,0.689435,0.305627,0.305627,0.631307,0.643839,0.591432,0.620820,0.294228,...,0.058824,0.164470,0.285376,0.298794,0.298794,0.506351,0.271018,0.675998,0.1,0.363636
9551212.0,0.543601,0.592850,0.642413,0.506036,0.506036,0.700237,0.699920,0.700632,0.699028,0.393749,...,0.352941,0.053021,0.095026,0.351339,0.351339,0.402679,0.392128,0.546586,0.0,0.000000
53300217.0,0.357407,0.754099,0.792663,0.503597,0.503597,0.769008,0.769739,0.755840,0.766575,0.505272,...,0.176471,0.127510,0.386620,0.656570,0.656570,0.398732,0.611050,0.683821,0.0,0.148601
1254489.0,0.391290,0.680341,0.733171,0.487277,0.487277,0.766757,0.771425,0.748083,0.765301,0.446943,...,0.352941,0.085705,0.527541,0.179448,0.179448,0.304347,0.202259,0.550187,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53382587.0,0.787807,0.402068,0.465878,0.192602,0.192602,0.482915,0.487917,0.461172,0.478569,0.173368,...,0.058824,0.162644,0.305030,0.130610,0.130610,0.361507,0.131689,0.571726,0.0,0.347619
53301482.0,0.257400,0.817294,0.906624,0.646353,0.646353,0.891583,0.894627,0.875231,0.893085,0.738536,...,0.000000,0.136722,0.573970,0.371170,0.371170,0.438681,0.375069,0.564328,0.0,0.250000
53301383.0,0.708092,0.436793,0.482028,0.179307,0.179307,0.580561,0.618644,0.480704,0.534047,0.197156,...,0.176471,0.269941,0.303194,0.479643,0.479643,0.349459,0.429846,0.534839,0.1,0.214286


In [12]:
Canonical_smis=Pubchem_data['Canonical_smiles']

In [13]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_all=aDesc.get_output()

In [14]:
Data_all=pd.DataFrame(Data_all)
Desc_names_names=aDesc.get_output_descriptors()
Data_all.columns=Desc_names_names
Data_all.index=Pubchem_data.index

In [15]:
X_test=Data_all[X_scaled_data.columns]
X_test.head()

,ESOL,LOGPcons,LOGP99,MRcons,MR99,SM5_B(p),SM6_B(p),SM3_B(p),SM4_B(p),H_Dz(p),...,C-006,Chi_Dz(p),Eta_D_epsiB,Eig01_AEA(ed),SpMax_AEA(ed),GATS7e,SpDiam_AEA(ri),MATS1i,F06[N-N],s3_phRelSize_2
cid,,,,,,,,,,,,,,,,,,,,,
5273,-1.602902,0.258130,-0.3453,53.1514,53.1514,7.205549,8.465290,4.832360,6.094002,36.564879,...,1.0,0.342638,0.022393,6.798627,6.798627,1.016172,5.231069,-0.436922,0.0,0.500000
13463,-1.651504,-0.288375,-0.7335,66.3790,66.3790,7.391460,8.639362,5.032571,6.284537,48.415156,...,1.0,0.259901,0.024606,6.923487,6.923487,0.978900,5.328745,-0.381000,0.0,0.362500
39148,-1.101717,-0.841930,-1.4107,74.2130,74.2130,7.435133,8.678132,5.089058,6.335432,53.766255,...,1.0,0.236753,0.016385,6.993002,6.993002,0.774649,5.365626,-0.523113,2.0,0.363095
56328,-1.754098,0.269420,0.0264,60.2442,60.2442,7.324406,8.575280,4.960958,6.218006,41.812397,...,1.0,0.259825,0.024683,6.864180,6.864180,0.992901,5.280229,-0.329590,0.0,0.250000
67358,-0.287497,-2.075408,-2.7416,63.9098,63.9098,7.406374,8.666884,5.028495,6.301740,47.075177,...,1.0,0.243159,0.023954,7.017898,7.017898,1.045927,5.380887,-0.336745,0.0,0.475000


In [16]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7279 Moleculars × 1416 Descriptors


,ESOL,LOGPcons,LOGP99,MRcons,MR99,SM5_B(p),SM6_B(p),SM3_B(p),SM4_B(p),H_Dz(p),...,C-006,Chi_Dz(p),Eta_D_epsiB,Eig01_AEA(ed),SpMax_AEA(ed),GATS7e,SpDiam_AEA(ri),MATS1i,F06[N-N],s3_phRelSize_2
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.765353,0.423157,0.440552,0.063323,0.063323,0.388992,0.433983,0.290492,0.354554,0.087353,...,0.058824,0.352000,0.155930,0.298344,0.298344,0.426964,0.281306,0.247943,0.0,0.500000
13463,0.760405,0.382023,0.416233,0.134994,0.134994,0.470704,0.505120,0.392909,0.445456,0.133558,...,0.058824,0.243607,0.171340,0.335329,0.335329,0.409370,0.347913,0.316906,0.0,0.362500
39148,0.816376,0.340358,0.373811,0.177441,0.177441,0.489899,0.520964,0.421804,0.469737,0.154422,...,0.058824,0.213281,0.114098,0.355920,0.355920,0.312955,0.373063,0.141651,0.2,0.363095
56328,0.749961,0.424007,0.463836,0.101754,0.101754,0.441233,0.478932,0.356276,0.413715,0.107813,...,0.058824,0.243507,0.171876,0.317761,0.317761,0.415979,0.314829,0.380307,0.0,0.250000
67358,0.899267,0.247517,0.290439,0.121615,0.121615,0.477259,0.516368,0.390824,0.453663,0.128333,...,0.058824,0.221674,0.166802,0.363294,0.363294,0.441010,0.383470,0.371483,0.0,0.475000
